In [1]:
# filter some warning messages
import warnings
warnings.filterwarnings("ignore")

In [2]:
import xarray as xr
import pandas as pd
import numpy as np

In [3]:
import gcsfs
# this only needs to be created once
gcs = gcsfs.GCSFileSystem(token='anon')

In [5]:
import intake

In [6]:
cat = intake.open_esm_datastore("https://raw.githubusercontent.com/intake/intake-esm/main/tutorial-catalogs/GOOGLE-CMIP6.json")
cat

PydanticUserError: If you use `@root_validator` with pre=False (the default) you MUST specify `skip_on_failure=True`. Note that `@root_validator` is deprecated and should be replaced with `@model_validator`.

For further information visit https://errors.pydantic.dev/2.0.3/u/root-validator-pre-skip

In [7]:
from intake import open_esm_datastore
col = open_esm_datastore("https://storage.googleapis.com/cmip6/pangeo-cmip6.json", skip_on_failure=True)

PydanticUserError: If you use `@root_validator` with pre=False (the default) you MUST specify `skip_on_failure=True`. Note that `@root_validator` is deprecated and should be replaced with `@model_validator`.

For further information visit https://errors.pydantic.dev/2.0.3/u/root-validator-pre-skip

In [5]:
cat_ts = col.search(variable_id = ['thetao', 'so'], experiment_id = 'piControl')
name_list_ts = cat_ts.df.source_id.unique()

In [6]:
cat_si = col.search(variable_id = ['siconc'], experiment_id = 'piControl')
cat_si.df.to_csv("dataset_si_raw.csv")

In [9]:
cat_t = col.search(variable_id = ['thetao'], experiment_id = 'piControl')
cat_t.df.to_csv("dataset_t_raw.csv")

cat_s = col.search(variable_id = ['so'], experiment_id = 'piControl')
cat_s.df.to_csv("dataset_s_raw.csv")

In [19]:
si_df = pd.read_csv('dataset_si_edited.csv')
s_df = pd.read_csv('dataset_s_edited.csv')
t_df = pd.read_csv('dataset_t_edited.csv')

In [44]:
coords_df = pd.read_csv('coords1.csv')

In [21]:
s_df = s_df.drop(labels=['institution_id', 'experiment_id', 'dcpp_init_year', 'table_id'], axis=1)
t_df = t_df.drop(labels=['institution_id', 'experiment_id', 'dcpp_init_year', 'table_id'], axis=1)
si_df = si_df.drop(labels=['dcpp_init_year'], axis=1)

In [22]:
st_df = pd.merge(s_df, t_df, how='inner', on='source_id')
n_st_df = st_df.drop(labels=['variable_id_x', 'member_id_x', 'grid_label_x', 'variable_id_y'], axis=1)
new_st_df = n_st_df.rename(columns={"zstore_x": "zstore_so", "zstore_y": "zstore_thetao"})

In [23]:
all_df = pd.merge(si_df, new_st_df, how='left', on='source_id')

In [24]:
nall_df = all_df.drop(labels=['variable_id', 'experiment_id', 'table_id', 'member_id_y', 'grid_label_y'], axis=1)
new_all_df = nall_df.rename(columns={"zstore": "zstore_siconc"})

In [24]:
new_all_df.to_csv('datase_all.csv')

In [14]:
new_all_df

,institution_id,source_id,member_id,grid_label,zstore_siconc,version,zstore_so,version_x,zstore_thetao,version_y
0,AS-RCEC,TaiESM1,r1i1p1f1,gn,gs://cmip6/CMIP6/CMIP/AS-RCEC/TaiESM1/piContro...,20200309,gs://cmip6/CMIP6/CMIP/AS-RCEC/TaiESM1/piContro...,20210213.0,gs://cmip6/CMIP6/CMIP/AS-RCEC/TaiESM1/piContro...,20210213.0
1,AWI,AWI-ESM-1-1-LR,r1i1p1f1,gn,gs://cmip6/CMIP6/CMIP/AWI/AWI-ESM-1-1-LR/piCon...,20200212,gs://cmip6/CMIP6/CMIP/AWI/AWI-ESM-1-1-LR/piCon...,20200212.0,gs://cmip6/CMIP6/CMIP/AWI/AWI-ESM-1-1-LR/piCon...,20200212.0
2,BCC,BCC-CSM2-MR,r1i1p1f1,gn,gs://cmip6/CMIP6/CMIP/BCC/BCC-CSM2-MR/piContro...,20200219,gs://cmip6/CMIP6/CMIP/BCC/BCC-CSM2-MR/piContro...,20181015.0,gs://cmip6/CMIP6/CMIP/BCC/BCC-CSM2-MR/piContro...,20181015.0
3,BCC,BCC-ESM1,r1i1p1f1,gn,gs://cmip6/CMIP6/CMIP/BCC/BCC-ESM1/piControl/r...,20200219,gs://cmip6/CMIP6/CMIP/BCC/BCC-ESM1/piControl/r...,20181218.0,gs://cmip6/CMIP6/CMIP/BCC/BCC-ESM1/piControl/r...,20181218.0
4,CAMS,CAMS-CSM1-0,r1i1p1f1,gn,gs://cmip6/CMIP6/CMIP/CAMS/CAMS-CSM1-0/piContr...,20190729,gs://cmip6/CMIP6/CMIP/CAMS/CAMS-CSM1-0/piContr...,20190729.0,gs://cmip6/CMIP6/CMIP/CAMS/CAMS-CSM1-0/piContr...,20190729.0
5,CAS,FGOALS-g3,r1i1p1f1,gn,gs://cmip6/CMIP6/CMIP/CAS/FGOALS-g3/piControl/...,20191030,gs://cmip6/CMIP6/CMIP/CAS/FGOALS-g3/piControl/...,20191126.0,gs://cmip6/CMIP6/CMIP/CAS/FGOALS-g3/piControl/...,20191126.0
6,CCCma,CanESM5,r1i1p1f1,gn,gs://cmip6/CMIP6/CMIP/CCCma/CanESM5/piControl/...,20190429,gs://cmip6/CMIP6/CMIP/CCCma/CanESM5/piControl/...,20190429.0,gs://cmip6/CMIP6/CMIP/CCCma/CanESM5/piControl/...,20190429.0
7,CCCma,CanESM5-CanOE,r1i1p2f1,gn,gs://cmip6/CMIP6/CMIP/CCCma/CanESM5-CanOE/piCo...,20190429,gs://cmip6/CMIP6/CMIP/CCCma/CanESM5-CanOE/piCo...,20190429.0,gs://cmip6/CMIP6/CMIP/CCCma/CanESM5-CanOE/piCo...,20190429.0
8,CMCC,CMCC-CM2-SR5,r1i1p1f1,gn,gs://cmip6/CMIP6/CMIP/CMCC/CMCC-CM2-SR5/piCont...,20200616,gs://cmip6/CMIP6/CMIP/CMCC/CMCC-CM2-SR5/piCont...,20200616.0,gs://cmip6/CMIP6/CMIP/CMCC/CMCC-CM2-SR5/piCont...,20200616.0
9,CMCC,CMCC-ESM2,r1i1p1f1,gn,gs://cmip6/CMIP6/CMIP/CMCC/CMCC-ESM2/piControl...,20210304,gs://cmip6/CMIP6/CMIP/CMCC/CMCC-ESM2/piControl...,20210304.0,gs://cmip6/CMIP6/CMIP/CMCC/CMCC-ESM2/piControl...,20210304.0


In [45]:
all_coord_df = pd.merge(new_all_df, coords_df, how='left', on='source_id')

In [46]:
np.where((all_coord_df['grid_label_x'] != all_coord_df['grid_label_y']) & (~all_coord_df['grid_label_y'].isnull()))

(array([44]),)

In [47]:
np.where(all_coord_df['grid_label_y'].isnull())

(array([19, 21, 22, 23, 24, 26, 27, 39, 45]),)

In [48]:
all_coord_df = all_coord_df.drop(labels=['grid_label_y'], axis=1)
all_coord_df = all_coord_df.rename(columns={"grid_label_x": "grid_label"})

In [49]:
all_coord_df.to_csv('all_coord.csv')

In [31]:
all_coord_df.iloc[37]

institution_id                                                  MRI
source_id                                                MRI-ESM2-0
member_id                                                  r1i2p1f1
grid_label_x                                                     gn
zstore_siconc     gs://cmip6/CMIP6/CMIP/MRI/MRI-ESM2-0/piControl...
version                                                    20210311
zstore_so         gs://cmip6/CMIP6/CMIP/MRI/MRI-ESM2-0/piControl...
version_x                                                20200222.0
zstore_thetao     gs://cmip6/CMIP6/CMIP/MRI/MRI-ESM2-0/piControl...
version_y                                                20210311.0
grid_label_y                                                     gr
xname                                                           lon
yname                                                           lat
lonname                                                         NaN
latname                                         

In [27]:
all_coord_df

,institution_id,source_id,member_id,grid_label_x,zstore_siconc,version,zstore_so,version_x,zstore_thetao,version_y,grid_label_y,xname,yname,lonname,latname,zname
0,AS-RCEC,TaiESM1,r1i1p1f1,gn,gs://cmip6/CMIP6/CMIP/AS-RCEC/TaiESM1/piContro...,20200309,gs://cmip6/CMIP6/CMIP/AS-RCEC/TaiESM1/piContro...,20210213.0,gs://cmip6/CMIP6/CMIP/AS-RCEC/TaiESM1/piContro...,20210213.0,gn,i,j,longitude,latitude,lev
1,AWI,AWI-ESM-1-1-LR,r1i1p1f1,gn,gs://cmip6/CMIP6/CMIP/AWI/AWI-ESM-1-1-LR/piCon...,20200212,gs://cmip6/CMIP6/CMIP/AWI/AWI-ESM-1-1-LR/piCon...,20200212.0,gs://cmip6/CMIP6/CMIP/AWI/AWI-ESM-1-1-LR/piCon...,20200212.0,gn,NaN,NaN,NaN,NaN,depth
2,BCC,BCC-CSM2-MR,r1i1p1f1,gn,gs://cmip6/CMIP6/CMIP/BCC/BCC-CSM2-MR/piContro...,20200219,gs://cmip6/CMIP6/CMIP/BCC/BCC-CSM2-MR/piContro...,20181015.0,gs://cmip6/CMIP6/CMIP/BCC/BCC-CSM2-MR/piContro...,20181015.0,gn,lon,lat,longitude,latitude,lev
3,BCC,BCC-ESM1,r1i1p1f1,gn,gs://cmip6/CMIP6/CMIP/BCC/BCC-ESM1/piControl/r...,20200219,gs://cmip6/CMIP6/CMIP/BCC/BCC-ESM1/piControl/r...,20181218.0,gs://cmip6/CMIP6/CMIP/BCC/BCC-ESM1/piControl/r...,20181218.0,gn,lon,lat,longitude,latitude,lev
4,CAMS,CAMS-CSM1-0,r1i1p1f1,gn,gs://cmip6/CMIP6/CMIP/CAMS/CAMS-CSM1-0/piContr...,20190729,gs://cmip6/CMIP6/CMIP/CAMS/CAMS-CSM1-0/piContr...,20190729.0,gs://cmip6/CMIP6/CMIP/CAMS/CAMS-CSM1-0/piContr...,20190729.0,gn,i,j,longitude,latitude,lev
5,CAS,FGOALS-g3,r1i1p1f1,gn,gs://cmip6/CMIP6/CMIP/CAS/FGOALS-g3/piControl/...,20191030,gs://cmip6/CMIP6/CMIP/CAS/FGOALS-g3/piControl/...,20191126.0,gs://cmip6/CMIP6/CMIP/CAS/FGOALS-g3/piControl/...,20191126.0,gn,i,j,longitude,latitude,lev
6,CCCma,CanESM5,r1i1p1f1,gn,gs://cmip6/CMIP6/CMIP/CCCma/CanESM5/piControl/...,20190429,gs://cmip6/CMIP6/CMIP/CCCma/CanESM5/piControl/...,20190429.0,gs://cmip6/CMIP6/CMIP/CCCma/CanESM5/piControl/...,20190429.0,gn,i,j,longitude,latitude,lev
7,CCCma,CanESM5-CanOE,r1i1p2f1,gn,gs://cmip6/CMIP6/CMIP/CCCma/CanESM5-CanOE/piCo...,20190429,gs://cmip6/CMIP6/CMIP/CCCma/CanESM5-CanOE/piCo...,20190429.0,gs://cmip6/CMIP6/CMIP/CCCma/CanESM5-CanOE/piCo...,20190429.0,gn,i,j,longitude,latitude,lev
8,CMCC,CMCC-CM2-SR5,r1i1p1f1,gn,gs://cmip6/CMIP6/CMIP/CMCC/CMCC-CM2-SR5/piCont...,20200616,gs://cmip6/CMIP6/CMIP/CMCC/CMCC-CM2-SR5/piCont...,20200616.0,gs://cmip6/CMIP6/CMIP/CMCC/CMCC-CM2-SR5/piCont...,20200616.0,gn,i,j,longitude,latitude,lev
9,CMCC,CMCC-ESM2,r1i1p1f1,gn,gs://cmip6/CMIP6/CMIP/CMCC/CMCC-ESM2/piControl...,20210304,gs://cmip6/CMIP6/CMIP/CMCC/CMCC-ESM2/piControl...,20210304.0,gs://cmip6/CMIP6/CMIP/CMCC/CMCC-ESM2/piControl...,20210304.0,gn,i,j,longitude,latitude,lev


In [15]:
namelist = list(si_df.source_id)

In [ ]:
for n in namelist:
    newsi = si_df[si_df.source_id == n]
    news = s_df[s_df.source_id == n]
    if len(news) == 1:
        print(news)
    else:
        print(n)
        news_zstore = news.zstore.values
    newt = t_df[t_df.source_id == n]
    if len(newt) == 1:
        print(',')
    else:
        print(n)
    


In [27]:
test = si_df[si_df.source_id == 'ts']

In [29]:
test.zstore.values

array([], dtype=object)